In [ ]:
import sys
sys.path.append('/home/sfalkner/repositories/github/random_forest_run/build')
sys.path.append('/home/sfalkner/repositories/github/ConfigSpace')
sys.path.append('/home/sfalkner/repositories/github/fanova')

import numpy as np
import matplotlib.pyplot as plt

import ConfigSpace
import fanova
import fanova.visualizer as viz

In [ ]:
# load data
X_full = np.loadtxt('/home/sfalkner/repositories/github/random_forest_run/test_data_sets/online_lda_features.csv', delimiter=',')
y_full = np.loadtxt('/home/sfalkner/repositories/github/random_forest_run/test_data_sets/online_lda_responses.csv', delimiter=',')


#n_samples = X_full.shape[0]//2
n_samples = 128

indices = np.random.choice(X_full.shape[0], n_samples)

if n_samples < X_full.shape[0]:
    X=X_full[indices]
    y=y_full[indices]
else:
    X=X_full
    y=y_full

    
# note that one can specify a ConfigSpace here, but if none is provided, all variables are
# assumed to be continuous and the range is (min,max)
f = fanova.fANOVA(X,y,  n_trees=32,bootstrapping=True)

## Reproducing the plots from Frank's fANOVA paper

In [ ]:
for i in range(3):

    #compute ground truth
    gt = []

    unique_values = list(set(X_full[:,i]))
    unique_values.sort()

    for v in unique_values:
        indices = np.where(X_full[:,i] == v)
        gt.append((v,np.mean(y_full[indices]), np.var(y_full[indices])))

    gt = np.array(gt)

    plt.figure()
    mew  = np.linspace( np.min(X[:,i]), np.max(X[:,i]), 100)
    mew2 = np.array([f.marginal_mean_variance_for_values([i],[v]) for v in mew])

    m = mew2[:,0]
    s = np.sqrt(mew2[:,1])

    plt.plot(mew, m)
    plt.fill_between(mew, m-s, m+s, alpha=.3)

    #plt.errorbar(gt[:,0], gt[:,1], yerr=np.sqrt(gt[:,2]))
    plt.scatter(gt[:,0], gt[:,1])
    plt.ylabel('Perplexity')
    plt.xlabel('parameter {}'.format(i))

plt.show()

In [ ]:
v = viz.Visualizer(f, f.cs)

In [ ]:
v.plot_marginal(0)
v.plot_marginal(1)
v.plot_marginal(2)
v.plot_pairwise_marginal([0,2], resolution=50)

In [ ]:
print(f.trees_total_variance)

In [ ]:
v.plot_marginal(2)
f.set_cutoffs((0,1500))
print(f.trees_total_variance)

# here is how one can change the labels and stuff
v.plot_marginal(2, show=False)
plt.title('cutoffs are now {}'.format(f.cutoffs))
plt.ylabel('Perplexity')
plt.show()

## Quantifying importance

The method quantify_importance now returns a dictionary with all the individual and total importance of all the computed terms up to the specified combination of input dimensions. Below a short example:

In [ ]:
f.set_cutoffs((-np.inf, np.inf))            #just reset the cutoffs
print(f.trees_total_variance)
#f.set_cutoffs((0,2000))
print(f.trees_total_variance)

importance_dict = f.quantify_importance([0,1,2])    

In [ ]:
for k in sorted(list(importance_dict.keys()), key=lambda t: importance_dict[t]['individual importance'], reverse=True):
    print(k, importance_dict[k])

In [ ]:
print(np.sum([importance_dict[v]['individual importance'] for v in importance_dict]))
print(f.trees_total_variance)
print(f.V_U_individual)
for i in range(32):
    print(np.sum([f.V_U_individual[v_u][i] for v_u in f.V_U_individual]), f.trees_total_variance[i])

## What's still missing

- options like 'improvement over default' or 'improvement over percentile', although the functionality is there by stetting the cutoffs manually
- documentation for the new API
- comparison against the Java implementation

In [ ]:
sys.path.append('/home/sfalkner/repositories/github/fanova_old')
from pyfanova.fanova_from_csv import FanovaFromCSV

data=np.hstack([X_full,y_full[:,None]])
np.savetxt('/tmp/tmp_data.csv', data, delimiter=',')
f = FanovaFromCSV("/tmp/tmp_data.csv")
